### Criteria 1: Is paper an original research article

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Load environment variables from .env file
texts = []
for i in range(10):
    response = client.responses.create(
      model="gpt-4o",
      input=[
        {
          "role": "system",
          "content": [
            {
              "type": "input_text",
              "text": "---\n\n### Task: Generate High-Quality Instruction-Following Training Samples\n\nYou are a helpful assistant creating training data for fine-tuning a T5-style language model (e.g., FLAN-T5) to perform **instruction-based classification with justification**.\n\nGenerate diverse examples where each example contains:\n\n- An `input`: consisting of a **fixed** natural language instruction (asking whether the paper is an original research article) followed by a paragraph of text that might resemble an abstract or a short summary (~1200–1800 words).  \n- An `output`: either `\"Yes\"` or `\"No\"`, followed by a clear justification referencing details from the text.\n\n### Output Format\nEach training example should be output as a JSON object on a **single line**, following this structure:\n\n```json\n{\n  \"input\": \"<Instruction>\\n\\nText: <Brief paragraph>\",\n  \"output\": \"<Yes/No>. <Justification referencing the content>\"\n}\n```\n\n---\n\n### **Instruction** (fixed for every example)\n\n> **Is the paper an original research article?**\n\n---\n\n### Guidelines\n\n1. **Positive Class (\"Yes\"/\"relevant paper\")**  \n   - Original research articles typically mention data collection or experimental design, methods, and statistical analyses.  \n   - These signals indicate that the article is reporting new, original findings (rather than summarizing or commenting on others’ work).\n\n2. **Negative Class (\"No\"/\"irrelevant paper\")**  \n   - If the text highlights that it is a review, perspective, editorial, commentary, poster, preprint, or other non-original-research formats, it should be labeled \"No\".  \n   - Mentions of “review article,” “systematic review,” “perspective,” “editorial,” or “opinion piece” are common indicators that the paper is **not** an original research article.  \n\n3. **Justification**  \n   - The justification should explicitly point to the presence or absence of relevant research features (data collection, methods, stats) or mention of non-research formats (e.g., review, perspectives, commentary) in the provided text.\n\n4. Generate multiple examples with a balanced distribution of \"Yes\" and \"No\" answers.\n\n---\n\n### Example\n\n```json\n{\n  \"input\": \"Instruction: Is the paper an original research article?\\n\\nText: This study describes the recruitment of 250 participants from two hospitals, followed by a randomized clinical trial to test a novel treatment for chronic migraine. Details of statistical power calculations and methods for adverse event tracking are provided, indicating new data were collected and analyzed to address a specific hypothesis.\",\n  \"output\": \"Yes. The text clearly states the collection of new patient data, a clinical trial design, and statistical analysis, all indicative of original research.\"\n}\n```\n\n```json\n{\n  \"input\": \"Instruction: Is the paper an original research article?\\n\\nText: This article presents a systematic review of recent advances in migraine therapies, compiling data from 52 studies published in the last five years. While the authors discuss results and synthesize findings across multiple trials, they do not present new, original data.\",\n  \"output\": \"No. The paper is described as a systematic review, indicating it is summarizing existing findings rather than providing new experimental data.\"\n}\n```\n\nUse these guidelines to produce multiple diverse training examples in JSONL format."
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "input_text",
              "text": "### Begin Generating:\nNow generate 100 examples in this format.\nEach example should be output as its own JSON object on a single line (JSONL format)."
            }
          ]
        }
      ],
      text={
        "format": {
          "type": "json_schema",
          "name": "instruction_output_schema",
          "schema": {
            "type": "object",
            "required": [
              "samples"
            ],
            "properties": {
              "samples": {
                "type": "array",
                "items": {
                  "type": "object",
                  "required": [
                    "input",
                    "output"
                  ],
                  "properties": {
                    "input": {
                      "type": "string",
                      "description": "The instruction and context provided as input. Make sure this property contains text that is above 600 tokens long"
                    },
                    "output": {
                      "type": "string",
                      "description": "The label corresponding to the input, which also includes a justification."
                    }
                  },
                  "additionalProperties": False
                },
                "description": "A list of input texts with their corresponding outputs."
              }
            },
            "additionalProperties": False
          },
          "strict": True
        }
      },
      reasoning={},
      tools=[],
      temperature=1,
      max_output_tokens=16384,
      top_p=1,
      store=True
    )
    texts.append(response.output_text)
    i += 1
    print(f"Done with {i}th iteration")

### Criteria 2: Does the paper mention AD related biomarkers?

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

texts = []
for i in range(10):
  response = client.responses.create(
    model="gpt-4o",
    input=[
      {
        "role": "system",
        "content": [
          {
            "type": "input_text",
            "text": "---\n\n### Task: Generate High-Quality Instruction-Following Training Samples\n\nYou are a helpful assistant creating training data for fine-tuning a T5-style language model (e.g., FLAN-T5) to perform **instruction-based classification with justification**.\n\nGenerate hundreds of diverse examples where each example contains:\n\n- An `input`: consisting of a **fixed** natural language instruction (asking whether the paper’s main focus is Alzheimer’s Disease) followed by a paragraph of text (~300–600 words) resembling an abstract or summary.  \n- An `output`: `\"Yes\"` or `\"No\"`, followed by a clear justification referencing details from the text.\n\n### Output Format\nEach training example should be output as a JSON object on a **single line**, following this structure:\n\n```json\n{\n  \"input\": \"<Instruction>\\n\\nText: <Brief paragraph>\",\n  \"output\": \"<Yes/No>. <Justification referencing the content>\"\n}\n```\n\n---\n\n### **Instruction** (fixed for every example)\n\n> **Does the paper have Alzheimer’s Disease (AD) as its main focus?**\n\n---\n\n### Guidelines\n\n1. **Positive Class (\"Yes\"/\"relevant paper\")**  \n   - The text explicitly involves AD topics (diagnosis, treatment, pathology, etc.).  \n   - The study includes or is about AD patients, including those with Mild Cognitive Impairment at risk of progressing to AD.  \n   - The study references specific AD biomarkers (e.g., amyloid beta, tau).  \n\n2. **Negative Class (\"No\"/\"irrelevant paper\")**  \n   - The text does not mention Alzheimer’s Disease or focuses on other neurodegenerative conditions exclusively.  \n   - Only general biomarkers or other neurodegeneration markers are mentioned (e.g., generic inflammatory markers) without tying them specifically to AD.  \n\n3. **Justification**  \n   - The justification should explicitly point to the presence (or absence) of AD-related details or biomarkers and clarify why that leads to a “Yes” or “No” classification.\n\n4. Produce multiple examples with a balanced distribution of “Yes” and “No” answers.\n\n---\n\n### Example\n\n```json\n{\n  \"input\": \"Instruction: Does the paper have Alzheimer's Disease (AD) as its main focus?\\n\\nText: This study investigates the role of amyloid beta deposition in the hippocampus of patients diagnosed with early-stage Alzheimer’s Disease, assessing its correlation with cognitive decline using neuropsychological tests.\",\n  \"output\": \"Yes. The text explicitly examines amyloid beta in patients with Alzheimer's Disease, confirming AD is the primary focus.\"\n}\n```\n\n```json\n{\n  \"input\": \"Instruction: Does the paper have Alzheimer's Disease (AD) as its main focus?\\n\\nText: Researchers performed an analysis of general tauopathies in older individuals and recorded multiple markers of neurodegeneration. While tau proteins are mentioned, the investigation primarily centers on age-related cognitive decline across diverse conditions, with no specific emphasis on AD.\",\n  \"output\": \"No. Although tau is referenced, there is no clear focus on Alzheimer’s Disease; the study discusses a broader range of neurodegenerative conditions.\"\n}\n```\n\nUse these guidelines to produce multiple diverse training examples in JSONL format."
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "input_text",
            "text": "### Begin Generating:\nNow generate 100 examples in this format.\\nEach example should be output as its own JSON object on a single line (JSONL format)."
          }
        ]
      }
    ],
    text={
      "format": {
        "type": "json_schema",
        "name": "instruction_output_schema",
        "schema": {
          "type": "object",
          "required": [
            "samples"
          ],
          "properties": {
            "samples": {
              "type": "array",
              "items": {
                "type": "object",
                "required": [
                  "input",
                  "output"
                ],
                "properties": {
                  "input": {
                    "type": "string",
                    "description": "The instruction and context provided as input. Make sure this property contains text that is above 600 tokens long"
                  },
                  "output": {
                    "type": "string",
                    "description": "The label corresponding to the input, which also includes a justification."
                  }
                },
                "additionalProperties": False
              },
              "description": "A list of input texts with their corresponding outputs."
            }
          },
          "additionalProperties": False
        },
        "strict": True
      }
    },
    reasoning={},
    tools=[],
    temperature=1,
    max_output_tokens=16384,
    top_p=1,
    store=True
  )
  texts.append(response.output_text)
  i += 1
  print(f"Done with {i}th iteration")

### Criteria 3: What's the sample size of the study?

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Load environment variables from .env file
texts = []
for i in range(10):
    response = client.responses.create(
    model="gpt-4o",
    input=[
        {
        "role": "system",
        "content": [
            {
            "type": "input_text",
            "text": "---\n\n### Task: Generate High-Quality Instruction-Following Training Samples\n\nYou are a helpful assistant creating training data for fine-tuning a T5-style language model (e.g., FLAN-T5) to perform **instruction-based classification with justification**.\n\nGenerate diverse examples where each example contains:\n\n- An `input`: consisting of a **fixed** natural language instruction (asking whether, if the paper mentions statistical analysis, the sample size exceeds 50) followed by a paragraph of text (~1200–1800 words) resembling an abstract or methods summary.\n- An `output`: either `\"Yes\"` or `\"No\"`, followed by a clear justification referencing details from the text, particularly focusing on the statistical analysis and the reported sample size.\n\n### Output Format\n\nEach training example should be output as a JSON object on a **single line**, following this structure:\n\n```json\n{\n  \"input\": \"<Instruction>\\n\\nText: <Brief paragraph>\",\n  \"output\": \"<Yes/No>. <Justification referencing the content>\"\n}\n```\n\n---\n\n### **Instruction** (fixed for every example)\n\n> **If the paper mentions some kind of statistical analysis, does the sample size exceed 50 (i.e., n >= 50)?**\n\n---\n\n### Guidelines\n\n1. **Positive Class (\"Yes\"/\"relevant paper\")**  \n   - The text should indicate that statistical analysis is performed and that the human sample size is greater than or equal to 50 (e.g., \"n = 60\", \"n = 75\", etc.).  \n   - This indicates robust data collection supporting statistical inferences.\n\n2. **Negative Class (\"No\"/\"irrelevant paper\")**  \n   - The text might include statistical analysis but the sample size is below 50 (e.g., \"n = 30\", \"n = 45\").  \n   - Note: Although sample size can sometimes be considered in context with other strengths of the study, for this criterion the focus is to check the explicit numeric threshold. If the sample size does not meet the threshold, assign the negative class.\n\n3. **Justification**  \n   - Clearly state why the example was assigned \"Yes\" (e.g., citing a sample size such as \"n = 75\" and mentioning statistical tests) or \"No\" (e.g., stating that the sample size (e.g., \"n = 30\") is below the threshold).\n\n4. Produce multiple examples with a balanced distribution of “Yes” and “No” answers.\n\n---\n\n### Example\n\n```json\n{\n  \"input\": \"Instruction: If the paper mentions some kind of statistical analysis, does the sample size exceed 50 (i.e., n >= 50)?\\n\\nText: In this study, researchers conducted a comprehensive analysis of cognitive performance among elderly participants. A detailed statistical analysis, including multivariate regression and ANOVA tests, was performed on the collected data. The methods section specifies that the study recruited a total of 72 individuals from community centers across the city. Data collection included standardized cognitive assessments, and the statistical analysis was rigorously applied to validate the results.\",\n  \"output\": \"Yes. The text clearly states that statistical analysis was performed and the sample size was 72, which exceeds the threshold of 50.\"\n}\n```\n\n```json\n{\n  \"input\": \"Instruction: If the paper mentions some kind of statistical analysis, does the sample size exceed 50 (i.e., n >= 50)?\\n\\nText: This pilot study aimed to explore preliminary trends in memory decline. Statistical comparisons were made using t-tests and chi-square tests to analyze the collected data. However, the study only involved 35 participants, all of whom were recruited from a single outpatient clinic. Despite the promising analytical approach, the reported sample size remained under the benchmark of 50.\",\n  \"output\": \"No. Although statistical analysis was conducted, the study's sample size of 35 is below the threshold of 50.\"\n}\n```\n\nUse these guidelines to produce multiple diverse training examples in JSONL format."
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "input_text",
            "text": "### Begin Generating:\nNow generate 100 examples in this format.\nEach example should be output as its own JSON object on a single line (JSONL format)."
            }
        ]
        }
    ],
    text={
        "format": {
        "type": "json_schema",
        "name": "instruction_output_schema",
        "strict": True,
        "schema": {
            "type": "object",
            "required": [
            "samples"
            ],
            "properties": {
            "samples": {
                "type": "array",
                "items": {
                "type": "object",
                "required": [
                    "input",
                    "output"
                ],
                "properties": {
                    "input": {
                    "type": "string",
                    "description": "The instruction and context provided as input. Make sure this property contains text that is above 600 tokens long"
                    },
                    "output": {
                    "type": "string",
                    "description": "The label corresponding to the input, which also includes a justification."
                    }
                },
                "additionalProperties": False
                },
                "description": "A list of input texts with their corresponding outputs."
            }
            },
            "additionalProperties": False
        }
        }
    },
    reasoning={},
    tools=[],
    temperature=1,
    max_output_tokens=16384,
    top_p=1,
    store=True
    )
    texts.append(response.output_text)
    i += 1
    print(f"Done with {i}th iteration")

### Criteria 4: Does the paper look at proteins as biomarkers (not genes, nor transcripts nor fragments)?

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Load environment variables from .env file
texts = []
for i in range(10):
    response = client.responses.create(
    model="gpt-4o",
    input=[
        {
        "role": "system",
        "content": [
            {
            "type": "input_text",
            "text": "---\n\n### Task: Generate High-Quality Instruction-Following Training Samples\n\nYou are a helpful assistant creating training data for fine-tuning a T5-style language model (e.g., FLAN-T5) to perform **instruction-based classification with justification**.\n\nGenerate diverse examples where each example contains:\n\n- An `input`: consisting of a **fixed** natural language instruction (asking whether the paper looks at proteins as biomarkers, not genes/transcripts/fragments) followed by a paragraph of text (~1200–1800 words) resembling an abstract or methods summary.  \n- An `output`: either `\"Yes\"` or `\"No\"`, followed by a clear justification referencing details from the text.\n\n### Output Format\nEach training example should be output as a JSON object on a **single line**, following this structure:\n\n```json\n{\n  \"input\": \"<Instruction>\\n\\nText: <Brief paragraph>\",\n  \"output\": \"<Yes/No>. <Justification referencing the content>\"\n}\n```\n\n---\n\n### **Instruction** (fixed for every example)\n\n> **Does the paper look at proteins as biomarkers (not genes, nor transcripts nor fragments)?**\n\n---\n\n### Guidelines\n\n1. **Positive Class (\"Yes\"/\"relevant paper\")**  \n   - The text should include multiple references to proteins, such as \"protein\", \"amyloid\", \"tau\", or specific AD-related proteins (e.g., \"beta-amyloid\").  \n   - The central context should be focused on measuring or analyzing proteins as biomarkers.\n\n2. **Negative Class (\"No\"/\"irrelevant paper\")**  \n   - If the text primarily focuses on non-protein entities like genes, RNA, transcription, or fragments, it should be classified as “No”.  \n   - The presence of these terms multiple times and as central themes indicates a non-protein biomarker study.\n\n3. **Justification**  \n   - The justification should clearly state why the study is classified as \"Yes\" by referencing repeated mentions and focus on proteins.  \n   - Conversely, it should highlight if the study mentions genes, transcripts, or fragments as the primary focus, leading to a \"No\" classification.\n\n4. Generate multiple diverse examples with a balanced distribution of “Yes” and “No” answers.\n\n---\n\n### Example\n\n```json\n{\n  \"input\": \"Instruction: Does the paper look at proteins as biomarkers (not genes, nor transcripts nor fragments)?\\n\\nText: In this comprehensive study, researchers examined the levels of beta-amyloid and tau proteins in cerebrospinal fluid samples from patients diagnosed with Alzheimer’s Disease. Multiple protein assays were conducted to quantify these biomarkers, ensuring a detailed protein-based analysis. Although a brief mention of gene expression was noted, the bulk of the investigation centered on protein quantification through immunoassays and mass spectrometry techniques.\",\n  \"output\": \"Yes. The paper predominantly focuses on protein biomarkers like beta-amyloid and tau, with only minimal and non-central reference to gene expression.\"\n}\n```\n\n```json\n{\n  \"input\": \"Instruction: Does the paper look at proteins as biomarkers (not genes, nor transcripts nor fragments)?\\n\\nText: This study explored the genetic basis of neurodegeneration by analyzing RNA samples extracted from brain tissues. A series of transcriptomic analyses were performed using next-generation sequencing. The research primarily focused on identifying gene expression profiles and transcriptional changes, with only passing references to protein levels which were not central to the analysis.\",\n  \"output\": \"No. The central focus is on gene and transcript-level analysis, with proteins mentioned only tangentially, if at all.\"\n}\n```\n\nUse these guidelines to produce multiple diverse training examples in JSONL format."
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "input_text",
            "text": "### Begin Generating:\nNow generate 100 examples in this format.\nEach example should be output as its own JSON object on a single line (JSONL format)."
            }
        ]
        }
    ],
    text={
        "format": {
        "type": "json_schema",
        "name": "instruction_output_schema",
        "strict": True,
        "schema": {
            "type": "object",
            "required": [
            "samples"
            ],
            "properties": {
            "samples": {
                "type": "array",
                "items": {
                "type": "object",
                "required": [
                    "input",
                    "output"
                ],
                "properties": {
                    "input": {
                    "type": "string",
                    "description": "The instruction and context provided as input. Make sure this property contains text that is above 600 tokens long"
                    },
                    "output": {
                    "type": "string",
                    "description": "The label corresponding to the input, which also includes a justification."
                    }
                },
                "additionalProperties": False
                },
                "description": "A list of input texts with their corresponding outputs."
            }
            },
            "additionalProperties": False
        }
        }
    },
    reasoning={},
    tools=[],
    temperature=1,
    max_output_tokens=16384,
    top_p=1,
    store=True
    )
    texts.append(response.output_text)
    i += 1
    print(f"Done with {i}th iteration")

### Criteria 5: Does the paper include Fluids from Non-Clinical Models to perform its study?

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Load environment variables from .env file
texts = []
for i in range(10):
    response = client.responses.create(
    model="gpt-4o",
    input=[
        {
        "role": "system",
        "content": [
            {
            "type": "input_text",
            "text": "---\n\n### Task: Generate High-Quality Instruction-Following Training Samples\n\nYou are a helpful assistant creating training data for fine-tuning a T5-style language model (e.g., FLAN-T5) to perform **instruction-based classification with justification**.\n\nGenerate diverse examples where each example contains:\n\n- An `input`: consisting of a **fixed** natural language instruction (asking whether the paper includes fluids from non-clinical models to perform its study) followed by a paragraph of text (~1200–1800 words) resembling an abstract or methods summary.\n- An `output`: either `\"Yes\"` or `\"No\"`, followed by a clear justification referencing details from the text.\n\n### Output Format\nEach training example should be output as a JSON object on a **single line**, following this structure:\n\n```json\n{\n  \"input\": \"<Instruction>\\n\\nText: <Brief paragraph>\",\n  \"output\": \"<Yes/No>. <Justification referencing the content>\"\n}\n```\n\n---\n\n### **Instruction** (fixed for every example)\n\n> **Does the paper include Fluids from Non-Clinical Models to perform its study?**\n\n---\n\n### Guidelines\n\n1. **Positive Class (\"Yes\"/\"relevant paper\")**  \n   - The text should clearly indicate that the study uses fluids (e.g., cerebrospinal fluid, blood, serum, plasma) obtained from non-clinical models (typically animal studies).  \n   - Biomarkers measured in these fluids or references to their use (for example, in AD research) should confirm the relevance.\n\n2. **Negative Class (\"No\"/\"irrelevant paper\")**  \n   - If the text focuses on tissue samples—such as brain slices, biopsy samples, or histology—indicate that the study does not include fluids from non-clinical models.  \n   - Keywords to look out for include \"tissue,\" \"histology,\" or \"brain slice,\" which should lead to a \"No\" classification.\n\n3. **Justification**  \n   - The justification should clearly state why the study is classified as \"Yes\" (citing the use of fluids such as CSF, blood, serum, or plasma in animal models) or \"No\" (highlighting a focus on tissue samples).\n  \n4. Produce multiple diverse examples with a balanced distribution of “Yes” and “No” answers.\n\n---\n\n### Example\n\n```json\n{\n  \"input\": \"Instruction: Does the paper include Fluids from Non-Clinical Models to perform its study?\\n\\nText: In this experimental study, researchers collected cerebrospinal fluid (CSF) and blood samples from a well-established rodent model of Alzheimer’s Disease. The objective was to analyze the levels of beta-amyloid and tau proteins in these fluids using immunoassay techniques, clearly indicating a focus on fluid biomarkers. The study did not involve any tissue sample analysis, such as brain slice histology, to support its findings.\",\n  \"output\": \"Yes. The text specifically describes the collection and analysis of CSF and blood from an animal model, without including tissue samples, thereby meeting the criteria.\"\n}\n```\n\n```json\n{\n  \"input\": \"Instruction: Does the paper include Fluids from Non-Clinical Models to perform its study?\\n\\nText: This study aimed to investigate the structural changes in brain tissue using detailed histological analyses. Researchers examined brain slices obtained from animal models and performed staining to highlight areas of neurodegeneration. Although there was a brief mention of serum in the introduction, the primary focus of the work was on tissue sample analysis through histology.\",\n  \"output\": \"No. Despite a brief mention of serum, the core of the study is based on tissue sample analysis and histology, which does not meet the fluid-based criteria.\"\n}\n```\n\nUse these guidelines to produce multiple diverse training examples in JSONL format."
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "input_text",
            "text": "### Begin Generating:\nNow generate 100 examples in this format.\nEach example should be output as its own JSON object on a single line (JSONL format)."
            }
        ]
        }
    ],
    text={
        "format": {
        "type": "json_schema",
        "name": "instruction_output_schema",
        "strict": True,
        "schema": {
            "type": "object",
            "required": [
            "samples"
            ],
            "properties": {
            "samples": {
                "type": "array",
                "items": {
                "type": "object",
                "required": [
                    "input",
                    "output"
                ],
                "properties": {
                    "input": {
                    "type": "string",
                    "description": "The instruction and context provided as input. Make sure this property contains text that is above 600 tokens long"
                    },
                    "output": {
                    "type": "string",
                    "description": "The label corresponding to the input, which also includes a justification."
                    }
                },
                "additionalProperties": False
                },
                "description": "A list of input texts with their corresponding outputs."
            }
            },
            "additionalProperties": False
        }
        }
    },
    reasoning={},
    tools=[],
    temperature=1,
    max_output_tokens=16384,
    top_p=1,
    store=True
    )
    texts.append(response.output_text)
    i += 1
    print(f"Done with {i}th iteration")

### Criteria 6: If the term Blood occurs in the paper, does it use Blood as an AD biomarker?

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Load environment variables from .env file
texts = []
for i in range(10):
    response = client.responses.create(
    model="gpt-4o",
    input=[
        {
        "role": "system",
        "content": [
            {
            "type": "input_text",
            "text": "---\n\n### Task: Generate High-Quality Instruction-Following Training Samples\n\nYou are a helpful assistant creating training data for fine-tuning a T5-style language model (e.g., FLAN-T5) to perform **instruction-based classification with justification**.\n\nGenerate diverse examples where each example contains:\n\n- An `input`: consisting of a **fixed** natural language instruction (asking if the paper uses blood as an AD biomarker) followed by a paragraph of text (~1200–1800 words) resembling an abstract or methods summary.\n- An `output`: either `\"Yes\"` or `\"No\"`, followed by a clear justification referencing specific details from the text.\n\n### Output Format\nEach training example should be output as a JSON object on a **single line**, following this structure:\n\n```json\n{\n  \"input\": \"<Instruction>\\n\\nText: <Brief paragraph>\",\n  \"output\": \"<Yes/No>. <Justification referencing the content>\"\n}\n```\n\n---\n\n### **Instruction** (fixed for every example)\n\n> **If the term Blood occurs in the paper, does it use Blood as an AD biomarker?**\n\n---\n\n### Guidelines\n\n1. **Positive Class (\"Yes\"/\"relevant paper\")**  \n   - The text should show that blood (or its derivatives, such as serum or plasma) is used to measure AD biomarkers (e.g., beta-amyloid, tau).  \n   - Indications may include phrases like \"blood samples were analyzed,\" \"serum was tested for AD biomarkers,\" or \"plasma levels of amyloid were measured.\"\n\n2. **Negative Class (\"No\"/\"irrelevant paper\")**  \n   - Exclude examples where \"blood\" is mentioned solely in the context of circulatory assessments such as \"blood pressure\" or studies focusing on vascular or hypertension aspects.  \n   - If the text includes keywords such as \"blood pressure,\" \"hypertension study,\" or \"vascular health,\" these should lead to a classification of \"No.\"\n\n3. **Justification**  \n   - The justification should clearly state the presence (or absence) of evidence that blood is used to sample AD biomarkers.  \n   - For positive examples, the justification must indicate that blood-based biomarker sampling (e.g., serum or plasma assays) is central to the study.  \n   - For negative examples, the justification should note that although blood is mentioned, it appears solely in contexts such as blood pressure or vascular assessments.\n\n4. Generate multiple diverse examples with a balanced distribution of “Yes” and “No” answers.\n\n---\n\n### Example\n\n```json\n{\n  \"input\": \"Instruction: If the term Blood occurs in the paper, does it use Blood as an AD biomarker?\\n\\nText: In this study, blood samples were drawn from patients to determine the levels of beta-amyloid and phosphorylated tau. The research focused on establishing a correlation between these biomarkers and disease progression in Alzheimer's Disease. The analysis of serum provided significant insights into the pathological changes characteristic of AD.\",\n  \"output\": \"Yes. The text clearly indicates that blood samples are used for biomarker analysis of beta-amyloid and tau, which are central to AD research.\"\n}\n```\n\n```json\n{\n  \"input\": \"Instruction: If the term Blood occurs in the paper, does it use Blood as an AD biomarker?\\n\\nText: The study examined the effects of high blood pressure on brain function among elderly participants. Blood pressure measurements were taken multiple times, and the focus was on assessing vascular health. Although blood was mentioned in the methods, there was no analysis of blood-based biomarkers related to Alzheimer's Disease.\",\n  \"output\": \"No. Although blood is mentioned, its role is confined to blood pressure measurement and vascular assessment rather than biomarker sampling for AD.\"\n}\n```\n\nUse these guidelines to produce multiple diverse training examples in JSONL format."
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "input_text",
            "text": "### Begin Generating:\nNow generate 100 examples in this format.\nEach example should be output as its own JSON object on a single line (JSONL format)."
            }
        ]
        }
    ],
    text={
        "format": {
        "type": "json_schema",
        "name": "instruction_output_schema",
        "schema": {
            "type": "object",
            "required": [
            "input",
            "output"
            ],
            "properties": {
            "input": {
                "type": "string",
                "description": "The instruction and context provided as input."
            },
            "output": {
                "type": "string",
                "description": "The label corresponding to the input, which may also include an optional justification."
            }
            },
            "additionalProperties": False
        },
        "strict": True
        }
    },
    reasoning={},
    tools=[],
    temperature=1,
    max_output_tokens=16384,
    top_p=1,
    store=True
    )
    texts.append(response.output_text)
    i += 1
    print(f"Done with {i}th iteration")